# Introduction

This notebook provides a set of functions and tools (confocal-tomography) to measure the tumor micro-environment within the blood brain niche model. It is designed to  analyze ND2 confocal images or 3D tif files.

Citation: Please refer to the citation notes for the related publication.

# Installation 

This notebook has only been tested with the following python packages:

python 3.8.1

jupyter notebook 5.3.4

jupyter lab 1.2.4

vtk 8.2.0

os - built into python

sys - built into python

numpy 1.17.3

pickle - built into python

math - built into python

ipywidgets 7.5.1

Ipython 7.11.1 - built into jupyter

pandas 0.25.3
 
matplotlib 3.1.2
    - mpl_toolkits part of above

scipy 1.3.2

functools - built into python

gc - built into python

seaborn 0.9.0

nd2reader 3.0.2

tifffile 2019.7.26.2

Usage:
Save ND2 file in a folder below this notebook called ../Experiment_###/Confocal/###.nd2. If this was your tenth experiment then the path would be ../Experiment_100/Confocal/100.nd2

Then use the notebook as annotated to analyze the data.


# Import libraries, custom classes/functions and setup the notebook

In [ ]:
import vtk # 3D analysis toolkit
from vtk.util import numpy_support #Enables conversion from vtk to numpy array
import os #access operating system functions
import sys
import numpy # helpes with matrix math
import numpy as np #Duplicate 
import pickle # encodes data into text files
import math # Math functions
from ipywidgets import interact
from IPython.display import clear_output
from IPython.display import Image
import pandas as pd # Data manipulation
from pandas import ExcelFile #
import matplotlib.pyplot as plt # plotting toolkit matplotlib (matlab)
from matplotlib.figure import Figure
import matplotlib.ticker as ticker # helps with plot tick control
from mpl_toolkits.mplot3d import Axes3D # 3D Plotting tool kit
import seaborn as sns # A plotting extension for matplotlib
import scipy.optimize # extends numpy for scientific analysis. fitting
import functools # helps run optimizatoin using function
import gc
from nd2reader import ND2Reader
from tifffile import imsave

def read_tiff(tiff_file):
    """
    tiff_file  : tif file input object
    dataImporter   : output vtkImage object
    reader         : output vtkTiff object
    ConstPixelDims : image dimensions
    """

    reader = vtk.vtkTIFFReader() 
    reader.SetFileName(tiff_file) 
    reader.Update() 
    castFilter = vtk.vtkImageCast()
    dataImporter = vtk.vtkImageCast()
    castFilter.SetInputConnection(reader.GetOutputPort())
    castFilter.SetOutputScalarTypeToUnsignedShort()
    castFilter.Update()
    dataImporter = castFilter.GetOutput()
    
    _extent = dataImporter.GetExtent()
    ConstPixelDims = [_extent[1]-_extent[0]+1, _extent[3]-_extent[2]+1, _extent[5]-_extent[4]+1]
    
    return dataImporter, reader, ConstPixelDims


def writeVtkPolyData(PolyData, file_name):
    """
    PolyData  : PolyData input object
    file_name   : output xml path file (example: 'out.vtp')
    """

    sg = vtk.vtkXMLPolyDataWriter()
    sg.SetFileName(file_name)
    sg.SetInputData(PolyData)
    sg.Write()

def Save_tiff_from_ND2(path,save_path,exp_num,shift_enabled,shift_num, field_direction, verbose):
    """
    path  : path of the nd2 file
    save_path   : the path to save the tiff files
    exp_num      : experiment number to save the image file name correctly
    shift_enabled : should the images overlap
    shift_num : what percentage should the images overlap
    field_direction : is the stitching vertical or horizontal?
    verbose : not used
    """
    with ND2Reader(path) as images:
        for colors, string in enumerate(images.metadata['channels']):
            x_width = images.metadata['width']
            y_height = images.metadata['height']
            z_levels = len(images.metadata['z_levels'])
            overlap = int(y_height*shift_num)
            frames = np.empty([x_width,y_height,z_levels])

            for fields in images.metadata['fields_of_view']:
                z_stack = np.empty([x_width, y_height])
                t=images.metadata['frames'][0]
                for z in images.metadata['z_levels']:
                    frame = np.array(images.get_frame_2D(colors,t,z,x_width,y_height,fields))
                    if z == 0:
                        z_stack = frame
                    else:
                        z_stack = np.dstack((z_stack,frame))

                if fields == 0:
                    frames = z_stack
                else:
                    if field_direction == True:
                        if shift_enabled:
                            overlap_z_stack = z_stack[overlap:,:,:]
                            frames = np.vstack((overlap_z_stack, frames))
                        else:
                            overlap_z_stack = z_stack[overlap:,:,:]
                            frames = np.vstack((frames, overlap_z_stack))
                    elif field_direction == False:
                        if shift_enabled:
                            overlap_z_stack = z_stack[:,overlap:,:]
                            frames = np.hstack((overlap_z_stack, frames))
                        else:
                            overlap_z_stack = z_stack[:,overlap:,:]
                            frames = np.hstack((frames, overlap_z_stack))
            frames = np.swapaxes(frames,0,2)#Rotates into page
            frames = np.swapaxes(frames,2,1)
            if verbose:
                print('Saving: ',exp_num, string)
            if string == 'FITC':
                imsave(save_path+str(exp_num)+'_GFP'+'.tif', frames)
            if string == 'Texas Red':
                imsave(save_path+str(exp_num)+'_RFP'+'.tif', frames)
            if string == 'DAPI':
                imsave(save_path+str(exp_num)+'_DAPI'+'.tif', frames)
            if string == 'Cy5':
                imsave(save_path+str(exp_num)+'_CY5'+'.tif', frames)
            if string == 'TD':
                imsave(save_path+str(exp_num)+'_TD'+'.tif', frames)

    
    
# Used to setup plotting parameters
font = {'sans-serif' : 'Arial',
        'family'   : 'sans-serif',
        'weight' : 'medium',
        'size'   : 10}

plt.rc('font', **font)

plt.rcParams['axes.autolimit_mode'] = 'round_numbers'
plt.rcParams['axes.xmargin'] = 0
plt.rcParams['axes.ymargin'] = 0


# Read the confocal z stack into memory

Load and read in a Nikon image acquisition file (.nd2)

In [ ]:
#set your file path here
base_filename = os.getcwd() +"\\"

#specify your experiment ID based on Experiment_tracker.xlsx
experiment = 1419

#organize your image files within the file path as Experiment_#/Confocal/
folder_filename = "Experiment_"+str(experiment).zfill(3)+"\\Confocal\\"

img_type = ".tif"
image_name_RFP = str(experiment)+"_RFP"
image_name_GFP = str(experiment)+"_GFP"
image_name_DAPI = str(experiment)+"_DAPI"
image_name_CY5 = str(experiment)+"_Cy5"

file_name_array=[]
file_name_array.append(base_filename+folder_filename+str(experiment)+"_RFP")#0
file_name_array.append(base_filename+folder_filename+str(experiment)+"_GFP")#1
file_name_array.append(base_filename+folder_filename+str(experiment)+"_CY5")#2
file_name_array.append(base_filename+folder_filename+str(experiment)+"_DAPI")#3

ConstPixelDims=[]
#Uncomment the next line is you want to save an ND2 file as tiff channels
#Save_tiff_from_ND2(base_filename+folder_filename+str(experiment)+".nd2",base_filename+folder_filename,experiment,False, 0.16, True, False)

GFP_file = (base_filename+folder_filename+image_name_GFP+img_type)
print(GFP_file)
if os.path.isfile(GFP_file):
    dataImporter_GFP, reader_GFP, ConstPixelDims = read_tiff(GFP_file)
    print("GFP Done")
else:
    print ("Reading GFP image files didn't work. Probably path name is wrong or file doesn't exist.")
    
RFP_file = (base_filename+folder_filename+image_name_RFP+img_type)
if os.path.isfile(RFP_file):
    dataImporter_RFP, reader_RFP, ConstPixelDims = read_tiff(RFP_file)
    print("RFP Done")
else:
    "Reading RFP image files didn't work. Probably path name is wrong or file doesn't exist."

CY5_file = (base_filename+folder_filename+image_name_CY5+img_type)
print(CY5_file)
if os.path.isfile(CY5_file):
    dataImporter_CY5, reader_CY5, ConstPixelDims = read_tiff(CY5_file)
    print("CY5 Done")
else:
    dataarray_CY5 = []
    "Reading CY5 image files didn't work. Probably path name is wrong or file doesn't exist."

DAPI_file = (base_filename+folder_filename+image_name_DAPI+img_type)
print(DAPI_file)

if os.path.isfile(DAPI_file):
    dataImporter_DAPI, reader_DAPI, ConstPixelDims = read_tiff(DAPI_file)
    print("DAPI Done")
else:
    dataarray_DAPI = []
    "Reading DAPI image files didn't work. Probably path name is wrong or file doesn't exist."
print(ConstPixelDims) # Prints the dimensions of the image based on the last file returned

# Visualize a render of the 3D confocal microscopic image

## Change opacity values for each microscope channel

In [ ]:
#Define properties that describe the alpha or blending channel of the images. Assumes 12 bit images.
#Adjust opacity values for each channel here
DAPI_alpha = 400
RFP_alpha = 400
GFP_alpha = 400
Cy5_alpha = 325

volOpacityDef = 0.25# Opacity of the different volumes (between 0.0 and 1.0)

#Cy5 channel
Cy5_alphaChannelFunc = vtk.vtkPiecewiseFunction()
Cy5_alphaChannelFunc.AddPoint(0, 0.0)
Cy5_alphaChannelFunc.AddPoint(Cy5_alpha, 0.0)
Cy5_alphaChannelFunc.AddPoint(Cy5_alpha+1, .5)
Cy5_alphaChannelFunc.AddPoint(1502, 0.5)
Cy5_alphaChannelFunc.AddPoint(4000, 1.0)

#DAPI channel
DAPI_alphaChannelFunc = vtk.vtkPiecewiseFunction()
DAPI_alphaChannelFunc.AddPoint(0, 0.0)
DAPI_alphaChannelFunc.AddPoint(DAPI_alpha, 0.0)
DAPI_alphaChannelFunc.AddPoint(DAPI_alpha+1, .5)
DAPI_alphaChannelFunc.AddPoint(1502, 0.5)
DAPI_alphaChannelFunc.AddPoint(4000, 1.0)

#RFP channel
RFP_alphaChannelFunc = vtk.vtkPiecewiseFunction()
RFP_alphaChannelFunc.AddPoint(0, 0.0)
RFP_alphaChannelFunc.AddPoint(RFP_alpha, 0.0)
RFP_alphaChannelFunc.AddPoint(RFP_alpha+1, .5)
RFP_alphaChannelFunc.AddPoint(2002, 0.5)
RFP_alphaChannelFunc.AddPoint(4000, 1.0)

#GFP channel
GFP_alphaChannelFunc = vtk.vtkPiecewiseFunction()
GFP_alphaChannelFunc.AddPoint(0, 0.0)
GFP_alphaChannelFunc.AddPoint(GFP_alpha, 0.0)
GFP_alphaChannelFunc.AddPoint(GFP_alpha+1, .5)
GFP_alphaChannelFunc.AddPoint(2000, 0.5)
GFP_alphaChannelFunc.AddPoint(4000, 1.0)

## Define render pseudocolors for each microscope channel
# This class stores color data and can create color tables from a few color points. For this demo, we want the three cubes
# to be of the colors red green and blue.

#Cy5 channel = pink
colorFunc_Cy5 = vtk.vtkColorTransferFunction()
colorFunc_Cy5.AddRGBPoint(100, 0.2, 0.0, 0.2)
colorFunc_Cy5.AddRGBPoint(500, 0.5, 0.0, 0.5)
colorFunc_Cy5.AddRGBPoint(1000, 0.75, 0.0, 0.75)
colorFunc_Cy5.AddRGBPoint(4000, 1.0, 0.0, 1.0)

#DAPI channel = blue
colorFunc_DAPI = vtk.vtkColorTransferFunction()
colorFunc_DAPI.AddRGBPoint(100, 0.0, 0.0, 0.02)
colorFunc_DAPI.AddRGBPoint(500, 0.0, 0.0, 0.5)
colorFunc_DAPI.AddRGBPoint(1000, 0.0, 0.0, 0.75)
colorFunc_DAPI.AddRGBPoint(4000, 0.0, 0.0, 1.0)

# GFP = green
colorFunc_GFP = vtk.vtkColorTransferFunction()
colorFunc_GFP.AddRGBPoint(100, 0.0, 0.02, 0.0)
colorFunc_GFP.AddRGBPoint(500, 0.0, 0.5, 0.0)
colorFunc_GFP.AddRGBPoint(1000, 0.0, 0.75, 0.0)
colorFunc_GFP.AddRGBPoint(4000, 0.0, 1.0, 0.0)

#RFP = red
RFP_colorFunc_RFP = vtk.vtkColorTransferFunction()
RFP_colorFunc_RFP.AddRGBPoint(0, 0.0, 0.0, 0.0)
RFP_colorFunc_RFP.AddRGBPoint(100, 0.05,0.0, 0.0)
RFP_colorFunc_RFP.AddRGBPoint(101, 0.05,0.0, 0.0)
RFP_colorFunc_RFP.AddRGBPoint(500, .25, 0.0, 0.0)
RFP_colorFunc_RFP.AddRGBPoint(1000, .5, 0.0, 0.0)
RFP_colorFunc_RFP.AddRGBPoint(4000, 1.0, 0.0, 0.0)

# The previous two classes stored properties. Because we want to apply these properties to the volume we want to render,
# we need to store them in a class that stores volume prpoperties.

#RFP 
RFP_volumeProperty = vtk.vtkVolumeProperty()
RFP_volumeProperty.SetColor(RFP_colorFunc_RFP)
RFP_volumeProperty.SetScalarOpacity(RFP_alphaChannelFunc)
RFP_volumeProperty.SetInterpolationTypeToLinear()
RFP_volumeProperty.SetDiffuse(50)
RFP_volumeProperty.SetAmbient(200)

#GFP
GFP_volumeProperty = vtk.vtkVolumeProperty()
GFP_volumeProperty.SetColor(colorFunc_GFP)
GFP_volumeProperty.SetScalarOpacity(GFP_alphaChannelFunc)
GFP_volumeProperty.SetInterpolationTypeToLinear()
GFP_volumeProperty.SetDiffuse(50)
GFP_volumeProperty.SetAmbient(200)

#DAPI
DAPI_volumeProperty = vtk.vtkVolumeProperty()
DAPI_volumeProperty.SetColor(colorFunc_DAPI)
DAPI_volumeProperty.SetScalarOpacity(DAPI_alphaChannelFunc)
DAPI_volumeProperty.SetInterpolationTypeToLinear()
DAPI_volumeProperty.SetDiffuse(50)
DAPI_volumeProperty.SetAmbient(200)

#Cy5
Cy5_volumeProperty = vtk.vtkVolumeProperty()
Cy5_volumeProperty.SetColor(colorFunc_Cy5)
Cy5_volumeProperty.SetScalarOpacity(Cy5_alphaChannelFunc)
Cy5_volumeProperty.SetInterpolationTypeToLinear()
Cy5_volumeProperty.SetDiffuse(50)
Cy5_volumeProperty.SetAmbient(200)

# Render size
width = 8000
height = 8000

# Define viewport ranges
xmins=[0,.5,0,.5]
xmaxs=[0.5,1,0.5,1]
ymins=[0,0,.5,.5]
ymaxs=[0.5,0.5,1,1]


## View a 3D rendering to verify the threshold are set correctly

In [ ]:
#RFP channel        
if os.path.isfile(RFP_file) & ('dataImporter_RFP' in globals()):
    RFP_volumeMapper = vtk.vtkGPUVolumeRayCastMapper()# This class describes how the volume is rendered (through ray tracing).
    RFP_volumeMapper.SetInputData(dataImporter_RFP)# We can finally create our volume. We also have to specify the data for it, as well as how the data will be rendered.
    RFP_volumeMapper.SetBlendModeToComposite()
    # The class vtkVolume is used to pair the preaviusly declared volume as well as the properties to be used when rendering that volume.
    RFP_volume = vtk.vtkVolume()
    RFP_volume.SetMapper(RFP_volumeMapper)
    RFP_volume.SetProperty(RFP_volumeProperty)

#GFP channel
if os.path.isfile(GFP_file) & ('dataImporter_GFP' in globals()):
    GFP_volumeMapper = vtk.vtkGPUVolumeRayCastMapper()# This class describes how the volume is rendered (through ray tracing).
    GFP_volumeMapper.SetInputData(dataImporter_GFP)
    GFP_volumeMapper.SetBlendModeToComposite()
    GFP_volume = vtk.vtkVolume()
    GFP_volume.SetMapper(GFP_volumeMapper)
    GFP_volume.SetProperty(GFP_volumeProperty)

#DAPI channel        
if os.path.isfile(DAPI_file) & ('dataImporter_DAPI' in globals()):
    DAPI_volumeMapper = vtk.vtkGPUVolumeRayCastMapper()
    DAPI_volumeMapper.SetInputData(dataImporter_DAPI)
    DAPI_volumeMapper.SetBlendModeToComposite()
    DAPI_volume = vtk.vtkVolume()
    DAPI_volume.SetMapper(DAPI_volumeMapper)
    DAPI_volume.SetProperty(DAPI_volumeProperty)

#Cy5        
if os.path.isfile(CY5_file) & ('dataImporter_CY5' in globals()):
    Cy5_volumeMapper = vtk.vtkGPUVolumeRayCastMapper()
    Cy5_volumeMapper.SetInputData(dataImporter_CY5)
    Cy5_volumeMapper.SetBlendModeToComposite()
    Cy5_volume = vtk.vtkVolume()
    Cy5_volume.SetMapper(Cy5_volumeMapper)
    Cy5_volume.SetProperty(Cy5_volumeProperty)
        

legendScaleActor = vtk.vtkLegendScaleActor()
# Ambient color light 
ColorLight = [1.0, 1.0, 0.0]
# Create a new vtkLight
light = vtk.vtkLight()
# Set its ambient color to yellow
light.SetAmbientColor(ColorLight)
# Set a 180 degree cone angle to avoid a spotlight-effect
light.SetConeAngle(180)
# Set its position to the sun's center
light.SetPosition(-300,245.5,500)  #May adjust based on your image size
# Set it as part of the scene (positional) and not following the camera
light.SetPositional(True)

renderer = vtk.vtkRenderer()

camera = renderer.MakeCamera()
camera.GetParallelProjection()
camera.SetPosition(-500, 1836/2, 0) #May adjust based on your image size
camera.SetFocalPoint(514/2, 1826/2, 0)  #May adjust based on your image size
camera.SetViewAngle(60.0)  #May adjust based on your image size
camera.SetRoll(-90.0)  #May adjust based on your image size

renderer.SetActiveCamera(camera)

#If we want to use the interactive renderer to manipulate the object
#Initialize the renderer and window, as well as creating a method for exiting the application
renderWin = vtk.vtkRenderWindow()
renderWin.AddRenderer(renderer)
renderInteractor = vtk.vtkRenderWindowInteractor()
renderInteractor.SetRenderWindow(renderWin)
style = vtk.vtkInteractorStyleTrackballCamera()

renderInteractor.SetInteractorStyle(style)

# We add the volume to the renderer. Comment out any channel that you don't want to include in the render.
#DAPI channel
if os.path.isfile(DAPI_file) & ('dataImporter_DAPI' in globals()):
    renderer.AddActor(DAPI_volume)
#GFP channel
if os.path.isfile(GFP_file) & ('dataImporter_GFP' in globals()):
    renderer.AddActor(GFP_volume)
#Cy5 channel
if os.path.isfile(CY5_file) & ('dataImporter_CY5' in globals()):
    renderer.AddActor(Cy5_volume)
#RFP channel
if os.path.isfile(RFP_file) & ('dataImporter_RFP' in globals()):
    renderer.AddActor(RFP_volume)

renderer.AddLight(light)

#set background color to white
renderer.SetBackground(255,255,255)
#set window size.
renderWin.SetSize(1000, 1000)

# A simple function to be called when the user decides to quit the application.
def exitCheck(obj, event):
    if obj.GetEventPending() != 0:
        obj.SetAbortRender(1)

# Tell the application to use the function as an exit check.
renderWin.AddObserver("AbortCheckEvent", exitCheck)
 
renderInteractor.Initialize()
# Because nothing will be rendered without any input, we order the first render manually before control is handed over to the main-loop.
renderWin.Render()
renderInteractor.Start()

After identifying the optimal opacity values, record them in Experiment_tracker.xlsx

# Convert voxel image into a traingular mesh and save as a VTK file

This step will convert the voxelized image into a mesh using the marching cubes algorithm and then save that file as a polydata format. This is useful later on for measuring cellular features.

## DAPI VTK

In [ ]:
%%time
dmc_DAPI = vtk.vtkMarchingCubes()
dmc_DAPI.SetInputConnection(reader_DAPI.GetOutputPort())
dmc_DAPI.ComputeNormalsOn()
dmc_DAPI.ComputeGradientsOn()
dmc_DAPI.SetValue(0,DAPI_alpha)
dmc_DAPI.Update()
result_DAPI = dmc_DAPI.GetOutput()

writeVtkPolyData(result_DAPI, base_filename+folder_filename+str(experiment)+"_DAPI_Polydata.vtk")

## GFP VTK

In [ ]:
%%time
dmc_GFP = vtk.vtkMarchingCubes()
dmc_GFP.SetInputConnection(reader_GFP.GetOutputPort())
dmc_GFP.ComputeNormalsOn()
dmc_GFP.ComputeGradientsOn()
dmc_GFP.SetValue(0,GFP_alpha)
dmc_GFP.Update()
result_GFP = dmc_GFP.GetOutput()

writeVtkPolyData(result_GFP, base_filename+folder_filename+str(experiment)+"_GFP_Polydata.vtk")

## RFP VTK

In [ ]:
%%time
dmc_RFP = vtk.vtkMarchingCubes()
dmc_RFP.SetInputConnection(reader_RFP.GetOutputPort())
dmc_RFP.ComputeNormalsOn()
dmc_RFP.ComputeGradientsOn()
dmc_RFP.SetValue(0,RFP_alpha)
dmc_RFP.Update()
result_RFP = dmc_RFP.GetOutput()

writeVtkPolyData(result_RFP, base_filename+folder_filename+str(experiment)+"_RFP_Polydata.vtk")

## Cy5 VTK

In [ ]:
%%time
dmc_Cy5 = vtk.vtkMarchingCubes()
dmc_Cy5.SetInputConnection(reader_Cy5.GetOutputPort())
dmc_Cy5.ComputeNormalsOn()
dmc_Cy5.ComputeGradientsOn()
dmc_Cy5.SetValue(0,Cy5_alpha)
dmc_Cy5.Update()
result_Cy5 = dmc_Cy5.GetOutput()

writeVtkPolyData(result_Cy5, base_filename+folder_filename+str(experiment)+"_Cy5_Polydata.vtk")

## Import the VTK files if not recalculating

### DAPI VTK

In [ ]:
reader_DAPI = vtk.vtkXMLPolyDataReader()
reader_DAPI.SetFileName(base_filename+folder_filename+str(experiment)+"_DAPI_Polydata.vtk")
reader_DAPI.Update()
result_DAPI = reader_DAPI.GetOutput()

### GFP VTK

In [ ]:
reader_GFP = vtk.vtkXMLPolyDataReader()
reader_GFP.SetFileName(base_filename+folder_filename+str(experiment)+"_GFP_Polydata.vtk")
reader_GFP.Update()
result_GFP = reader_GFP.GetOutput()

### RFP VTK

In [ ]:
reader_RFP = vtk.vtkXMLPolyDataReader()
reader_RFP.SetFileName(base_filename+folder_filename+str(experiment)+"_RFP_Polydata.vtk")
reader_RFP.Update()
result_RFP = reader_RFP.GetOutput()

### Cy5 VTK

In [ ]:
reader_Cy5 = vtk.vtkXMLPolyDataReader()
reader_Cy5.SetFileName(base_filename+folder_filename+str(experiment)+"_Cy5_Polydata.vtk")
reader_Cy5.Update()
result_Cy5 = reader_Cy5.GetOutput()

# Analyze the RFP channel (endothelial barrier) 

Obtain the RFP centroids

In [ ]:
conn_RFP = vtk.vtkPolyDataConnectivityFilter()
conn_RFP.SetInputData(result_RFP)
conn_RFP.SetExtractionModeToAllRegions()
conn_RFP.Update()
nregions_RFP = conn_RFP.GetNumberOfExtractedRegions()
print(nregions_RFP)
conn_RFP.SetExtractionModeToSpecifiedRegions()
conn_RFP.Update()
regionsizes_RFP = conn_RFP.GetRegionSizes ()
object_data_RFP = []

print("ID    Centroid(um)")
for region_RFP in range(nregions_RFP):
    mass_RFP = regionsizes_RFP.GetValue(region_RFP)
    if mass_RFP < 1500 or mass_RFP > 100000 :
        continue
    conn_RFP.InitializeSpecifiedRegionList()
    conn_RFP.AddSpecifiedRegion(region_RFP)
    conn_RFP.Update()

    p_RFP = vtk.vtkPolyData()
    p_RFP = conn_RFP.GetOutput()
    extent_tuple_RFP = p_RFP.GetBounds()
    center_RFP = ((extent_tuple_RFP[0]+(extent_tuple_RFP[1]-extent_tuple_RFP[0])/2),(extent_tuple_RFP[2]+(extent_tuple_RFP[3]-extent_tuple_RFP[2])/2),(extent_tuple_RFP[4]+(extent_tuple_RFP[5]-extent_tuple_RFP[4])/2))
    object_data_RFP.append([region_RFP,center_RFP])
    center_RFP_formatted = [ '%.3f' % elem for elem in center_RFP ]
    print(region_RFP,center_RFP_formatted)
print("Done")

with open( base_filename+folder_filename+str(experiment)+"_RFP_Data.txt", "wb") as fp:
    pickle.dump(object_data_RFP, fp)

## Read in saved RFP centroids

In [ ]:
with open(base_filename+folder_filename+str(experiment)+"_RFP_Data.txt", "rb") as fp:   # Unpickling
    object_data_RFP = pickle.load(fp)
print(base_filename+folder_filename+str(experiment)+"_RFP_Data.txt")

## Fit a plane to the RFP centroids (endothelial barrier)

### Define parameters and plane formation

In [ ]:
%matplotlib widget
font = {'sans-serif' : 'Arial',
        'family'   : 'sans-serif',
        'weight' : 'medium',
        'size'   : 10}

plt.rc('font', **font)
def plane(x, y, params):
    a = params[0]
    b = params[1]
    c = params[2]
    z = a*x + b*y + c
    return z

def error(params, points):
    diff_result = 0
    SStot_result = 0
    z_avg = sum([item[2] for item in points])/float(len(points))
    #print(z_avg)
    for (x,y,z) in points:
        plane_z = plane(x, y, params)
        diff = abs(z - plane_z)
        SStot = abs(z - z_avg)
        diff_result += diff**2
        SStot_result += SStot**2
    return diff_result

def cross(a, b):
    return [a[1]*b[2] - a[2]*b[1],
            a[2]*b[0] - a[0]*b[2],
            a[0]*b[1] - a[1]*b[0]]

def graph(formula, x_range):  
    x = np.array(x_range)  
    y = eval(formula)
    plt.plot(x, y)  
    plt.show()
    
points = [item[1] for item in object_data_RFP]

fun = functools.partial(error, points=points)
params0 = [0, 0, 0]
res = scipy.optimize.minimize(fun, params0)

a = res.x[0]
b = res.x[1]
c = res.x[2]
final_z = a*1+b*1+c*1
xs, ys, zs = zip(*points)

point  = np.array([0.0, 0.0, c])
normal = np.array(cross([1,0,a], [0,1,b]))
d = -point.dot(normal)
print("c:",-d,"d:",d)

fig = plt.figure(num = None, figsize = (4,4),dpi = 150, facecolor = 'w', edgecolor = None, frameon=None)
ax = fig.add_subplot(111, projection='3d')

xx, yy = np.meshgrid([-5,ConstPixelDims[0]], [-5,ConstPixelDims[1]])
ax.auto_scale_xyz
ax.set_xlim(0,ConstPixelDims[0])
ax.set_ylim(0,ConstPixelDims[1])
ax.set_zlim(  0, 75)
ax.set_xlabel('X (px)')
ax.set_ylabel('Y (px)')
ax.set_zlabel('Z (px)')
ax.dist = 13
import matplotlib.ticker as plticker
xloc = plticker.MultipleLocator(base=250.0) 
ax.xaxis.set_major_locator(xloc)
yloc = plticker.MultipleLocator(base=1500.0) 
ax.yaxis.set_major_locator(yloc)
zloc = plticker.MultipleLocator(base=25.0) 
ax.zaxis.set_major_locator(zloc)
plt.tight_layout()

def pltplane(n1 = float(normal[0])*10000.0,n2 = float(normal[1])*10000.0,n3 = float(normal[2]),n4 = float(0)):
    """
    Function to fit a plane to the points
    n1-n4 are the normals initialization values corresponding to the four sliders called with this function.
    normal : output returns the normals
    """
    global normal
    global push_value
    t = []
    ax.clear()
    n1 = n1/10000.0
    n2 = n2/10000.0
    n3 = n3
    print(n1,n2,n3)
    for xp, yp, zp in zip(xs, ys, zs):
        if (((-n1 * xp - n2 * yp - d) * 1. /n3)+n4) < zp:
            t.append(1)
        else:
            t.append(0)
    z = ((-n1 * xx - n2 * yy - d) * 1. /n3)+n4
    ax.scatter(xs, ys, zs, c = t,s=4,lw=0.0)
    #ax.scatter(xr, yr, zr, marker = "s", c='green',s=4,lw=0.0)
    ax.plot_surface(xx, yy, z, alpha=0.2, color=[0,1,0])
    plt.show()
    normal = (n1,n2,n3)
    push_value = n4
    str1 = ' '.join(str(e) for e in normal)
    file = open(base_filename+folder_filename+str(experiment)+"_Normals.txt","w") 
    file.write(str1) 
    file.close() 
    return 

### Visualize RFP centroids and plane fit

In [ ]:
interact(pltplane, n1 = (-100,100,0.01),n2 = (-100,100,0.01),n3 = (0,1.4,0.01),n4 = (-15,15,0.1))

record the plane fit values (c, d, normals, push) in Experiment_tracker.xlsx

# Analyze the remaining microscope channels for phenotypic descriptors

In [ ]:
def cell_analysis(image, path):
    """
    image  : the dataset to be analyzed
    thresh   : the opacity threshold used 
    path         : path where to save the measurements of each cell
    object_data : array of measurements for each cell in the image
    """
    global df_sub
    global xy_pxtoum
    global z_pxtoum
    global push_value
    global normal
    global min_size
    global d_value
    global c_value

    import csv

    conn = vtk.vtkPolyDataConnectivityFilter()
    conn.SetInputData(image)
    conn.SetExtractionModeToAllRegions()
    conn.Update()

    nregions = conn.GetNumberOfExtractedRegions()
    print(nregions)
    conn.SetExtractionModeToSpecifiedRegions()
    conn.Update()
    regionsizes = conn.GetRegionSizes ()

    #Create a plane
    planeSource = vtk.vtkPlane()

    planeSource.SetOrigin(0,0,c_value)
    planeSource.SetNormal(normal[0], normal[1], normal[2])
    planeSource.Push(push_value)

    polydata_collection = []
    object_data = []

    for region in range(nregions):

        conn.InitializeSpecifiedRegionList()
        conn.AddSpecifiedRegion(region)
        conn.Update()
        p = vtk.vtkPolyData()
        p = conn.GetOutput()

        massProp =vtk.vtkMassProperties()
        massProp.SetInputData(p)
        volume = massProp.GetVolume()#

        # Specify the cell size cutoff value. A volume < 100 is suitable for cell types ~ 15um diamter. volume < 50 is recommended for smaller cells
        if volume < min_size:
            continue
        extent_tuple = p.GetBounds()
        center = ((extent_tuple[0]+(extent_tuple[1]-extent_tuple[0])/2),(extent_tuple[2]+(extent_tuple[3]-extent_tuple[2])/2),(extent_tuple[4]+(extent_tuple[5]-extent_tuple[4])/2))
        
        # Calculate the amount of cell through the barrier and perform boolean operation
        clip = vtk.vtkClipPolyData()
        clip.SetInputConnection(conn.GetOutputPort())
        clip.SetClipFunction(planeSource)
        clip.SetValue(0)
        clip.Update()
        polyData = vtk.vtkPolyData()
        polyData = clip.GetOutput()
        fillHolesFilter =   vtk.vtkFillHolesFilter()
        fillHolesFilter.SetInputData(polyData)
        fillHolesFilter.SetHoleSize(10000.0)

        #Make the triangle winding order consistent
        normals = vtk.vtkPolyDataNormals()
        normals.SetInputConnection(fillHolesFilter.GetOutputPort())
        normals.ConsistencyOn()
        normals.SplittingOff()
        normals.Update()

        #Restore the original normals
        normals.GetOutput().GetPointData().SetNormals(polyData.GetPointData().GetNormals())
        polyData_filled = vtk.vtkPolyData()
        polyData_filled = normals.GetOutput()
        extent_tuple_cut = polyData_filled.GetBounds()
        center_cut = ((extent_tuple_cut[0]+(extent_tuple_cut[1]-extent_tuple_cut[0])/2),(extent_tuple_cut[2]+(extent_tuple_cut[3]-extent_tuple_cut[2])/2),(extent_tuple_cut[4]+(extent_tuple_cut[5]-extent_tuple_cut[4])/2))

        mass_cut = polyData_filled.GetNumberOfPolys()
        mass_pre_polys = p.GetNumberOfPolys()

        surface_area = massProp.GetSurfaceArea()
        normalized_shape_index = massProp.GetNormalizedShapeIndex()
        massProp_cut =vtk.vtkMassProperties()
        massProp_cut.SetInputData(polyData_filled)
        volume_cut = massProp_cut.GetVolume()

        ### Measure remaining metrics
        sphericity=(((math.pi**(1/3))*(6*volume)**(2/3))/surface_area)
        boundingboxratio = (volume**(1/3))/(extent_tuple[5]-extent_tuple[4])
        extravasated_distance = planeSource.DistanceToPlane(center)*z_pxtoum
        center_formatted = [ '%.3f' % elem for elem in center]
        center_cut_formatted = [ '%.3f' % elem for elem in center_cut]
        object_data.append([region,center, volume,volume_cut,((volume_cut/volume)*100), center_cut, sphericity, normalized_shape_index, boundingboxratio, extravasated_distance])

        print(region, center_formatted, "{0:.3f}".format(volume), extravasated_distance, sphericity)
    with open(path, "wb") as fp:   #Pickling
        pickle.dump(object_data, fp)
    return object_data

## Read in the Experiment_tracker information and analyze the channels that exist

In [ ]:
df = pd.read_excel(base_filename+"Experiment_tracker.xlsx", sheet_name='Experiment_Values')
df_sub = df.loc[df['ID'] == experiment]

DAPI_alpha = df_sub.iloc[0][20]
GFP_alpha = df_sub.iloc[0][9]
RFP_alpha = df_sub.iloc[0][10]
CY5_alpha = df_sub.iloc[0][19]
xy_pxtoum = df_sub.iloc[0][17]
z_pxtoum = df_sub.iloc[0][16]
push_value = df_sub.iloc[0][14]
min_size=df_sub.iloc[0][15] #The filter size for when to ignore an object in the VTK list.
c_value = df_sub.iloc[0][12]
d_value = -c

df_sub.style
normalstring = df_sub.iloc[0][11]
normalsubstring=normalstring[1:-1]
normals=np.fromstring(normalsubstring, dtype='float', sep=', ')
print("Experiment: ", experiment, "GFP alpha: ", GFP_alpha, "RFP alpha: ", RFP_alpha, "CY5 alpha: ", CY5_alpha, "DAPI alpha: ", DAPI_alpha, "XY conversion: ", xy_pxtoum, "Z conversion: ", z_pxtoum, "Min object size: ", min_size, "Normals: ", normal, "Push value: ", push_value,"D: ", d)

if os.path.isfile(GFP_file) & ('result_GFP' in globals()):
    list_of_objects_GFP = cell_analysis(result_GFP, base_filename+folder_filename+str(experiment)+"_GFP_Data.txt")
    print("GFP Done")
if os.path.isfile(DAPI_file) & ('result_DAPI' in globals()):
    list_of_objects_DAPI = cell_analysis(result_DAPI, base_filename+folder_filename+str(experiment)+"_DAPI_Data.txt")
    print("DAPI Done")
if os.path.isfile(CY5_file) & ('result_CY5' in globals()):
    list_of_objects_CY5 = cell_analysis(result_CY5, base_filename+folder_filename+str(experiment)+"_CY5_Data.txt")
    print("CY5 Done")

## Check that centroids were measured accuractely
### Centroids are rendered as points over the volume as a quality check

In [ ]:
def centroid_check(image, volumeProperty, path):
    """
    image  : path to the volume to be rendered below the centroid points
    volumeProperty   : description of how the volume should be rendered (set earlier in the notebook)
    path   : path to find centroids of the cells
    return : void
    """
    
    global df_sub
    global xy_pxtoum
    global z_pxtoum
    global push_value
    global normal
    global min_size
    global d_value
    global c_value
    global ConstPixelDims


    with open(path, "rb") as fp:  
        object_data = pickle.load(fp) 

    points_list = vtk.vtkPoints()
    cell_volume_data = [item[1] for item in object_data]
    for next_point in cell_volume_data:
        points_list.InsertNextPoint (next_point)

    pointsPolydata = vtk.vtkPolyData()
    pointsPolydata.SetPoints(points_list)
    vertexFilter = vtk.vtkVertexGlyphFilter()
    vertexFilter.SetInputData(pointsPolydata)
    vertexFilter.Update()

    point_polydata = vtk.vtkPolyData()
    point_polydata.ShallowCopy(vertexFilter.GetOutput())

    #cell_volume_data
    #Setup colors
    red = (255, 0, 0)
    green = (0, 255, 0)
    blue = (0, 0, 255)
    near_red = (125,0,0)
    colors =vtk.vtkUnsignedCharArray()
    colors.SetNumberOfComponents(3)
    colors.SetName("Colors")
    colors.InsertNextTuple(red)
    colors.InsertNextTuple(green)
    colors.InsertNextTuple(blue)

    #Visualization
    #GFP centroids
    point_mapper = vtk.vtkPolyDataMapper()
    point_mapper.SetInputData(point_polydata)
    actor_points = vtk.vtkActor()
    actor_points.SetMapper(point_mapper)
    actor_points.GetProperty().SetPointSize(10)

    # Create the GFP cell volume. We also have to specify the data for it, as well as how the data will be rendered.
    volumeMapper = vtk.vtkGPUVolumeRayCastMapper()# This class describes how the volume is rendered (through ray tracing).
    volumeMapper.SetInputData(image)
    volumeMapper.SetBlendModeToComposite()
    volume = vtk.vtkVolume()
    volume.SetMapper(volumeMapper)
    volume.SetProperty(volumeProperty)

    #Create a plane
    planeSource = vtk.vtkPlaneSource()
    planeSource.SetXResolution(4)
    planeSource.SetYResolution(4)
    planeSource.SetOrigin(0,0,c)
    planeSource.SetPoint1(ConstPixelDims[0]*4,0,0)
    planeSource.SetPoint2(0,ConstPixelDims[1]*4,0)
    planeSource.SetCenter(0,0,c)
    planeSource.SetNormal(normal[0], normal[1], normal[2])
    planeSource.Update()
    planeSource.Push(push_value)
    planeSource.Update()
    plane = vtk.vtkPolyData()
    plane = planeSource.GetOutput()

    #Create a mapper and actor
    pmapper = vtk.vtkPolyDataMapper()
    pmapper.SetInputData(plane)

    cone = vtk.vtkConeSource()
    cone.SetResolution(20)

    glyph = vtk.vtkGlyph3D()
    glyph.SetInputConnection(planeSource.GetOutputPort())
    glyph.SetSourceConnection(cone.GetOutputPort())
    glyph.SetVectorModeToUseNormal()
    glyph.SetScaleModeToScaleByVector()
    glyph.SetScaleFactor(50)

    spikeMapper = vtk.vtkPolyDataMapper()
    spikeMapper.SetInputConnection(glyph.GetOutputPort())

    spikeActor = vtk.vtkLODActor()
    spikeActor.SetMapper(spikeMapper)

    spikeActor.GetProperty().SetDiffuseColor(red)
    spikeActor.GetProperty().SetSpecular(.4)
    spikeActor.GetProperty().SetSpecularPower(20)

    actorp = vtk.vtkActor()
    actorp.SetMapper(pmapper)
    actorp.GetProperty().SetColor(0,0,255)
    actorp.GetProperty().SetLineWidth(2)
    actorp.GetProperty().SetOpacity(0.5)

    legendScaleActor = vtk.vtkLegendScaleActor()
    # Ambient color light 
    ColorLight = [1.0, 1.0, 0.0]
    # Create a new vtkLight
    light = vtk.vtkLight()
    # Set its ambient color to yellow
    light.SetAmbientColor(ColorLight)
    # Set a 180 degree cone angle to avoid a spotlight-effect
    light.SetConeAngle(180)
    # Set its position to the sun's center
    light.SetPosition(-300,245.5,500)
    # Set it as part of the scene (positional) and not following the camera
    light.SetPositional(True)

    renderer = vtk.vtkRenderer()

    camera = renderer.MakeCamera()
    camera.GetParallelProjection()
    camera.SetPosition(ConstPixelDims[0], ConstPixelDims[1]/2, c)
    camera.SetFocalPoint(ConstPixelDims[0]/2, ConstPixelDims[1]/2, c)
    camera.SetViewAngle(60.0)
    camera.SetRoll(270.0)
    renderer.SetActiveCamera(camera)
    renderWin = vtk.vtkRenderWindow()
    renderWin.AddRenderer(renderer)
    renderInteractor = vtk.vtkRenderWindowInteractor()
    renderInteractor.SetRenderWindow(renderWin)
    style = vtk.vtkInteractorStyleTrackballCamera()
    axes = vtk.vtkAxesActor()
    widget = vtk.vtkOrientationMarkerWidget()
    widget.SetOutlineColor( 0.9300, 0.5700, 0.1300 )
    widget.SetOrientationMarker( axes )
    widget.SetInteractor( renderInteractor )
    widget.SetViewport( 0.0, 0.0, 0.4, 0.4 )
    widget.SetEnabled( 1 )
    widget.InteractiveOn()

    renderInteractor.SetInteractorStyle(style)
    # We add the volume to the renderer ...
    renderer.AddVolume(volume)
    renderer.AddActor(actor_points)
    renderer.AddActor(actorp)
    renderer.AddLight(light)
    # ... set background color to black ...
    renderer.SetBackground(255,0,255)
    # ... and set window size.
    renderWin.SetSize(1000, 1000)

    # A simple function to be called when the user decides to quit the application.
    def exitCheck(obj, event):
        if obj.GetEventPending() != 0:
            obj.SetAbortRender(1)

    # Tell the application to use the function as an exit check.
    renderWin.AddObserver("AbortCheckEvent", exitCheck)

    renderInteractor.Initialize()
    # Because nothing will be rendered without any input, we order the first render manually before control is handed over to the main-loop.
    renderWin.Render()
    renderInteractor.Start()
    return 

if os.path.isfile(GFP_file) & ('dataImporter_RFP' in globals()):
    centroid_check(dataImporter_RFP, RFP_volumeProperty, base_filename+folder_filename+str(experiment)+"_RFP_Data.txt")
    print("RFP Done")
else:
    print("No RFP channel")
if os.path.isfile(GFP_file) & ('dataImporter_GFP' in globals()):
    centroid_check(dataImporter_GFP, GFP_volumeProperty, base_filename+folder_filename+str(experiment)+"_GFP_Data.txt")
    print("GFP Done")
else:
    print("No GFP channel")
if os.path.isfile(DAPI_file) & ('dataImporter_DAPI' in globals()):
    centroid_check(dataImporter_DAPI, DAPI_volumeProperty, base_filename+folder_filename+str(experiment)+"_DAPI_Data.txt")
    print("DAPI Done")
else:
    print("No DAPI channel")
if os.path.isfile(CY5_file) & ('dataImporter_CY5' in globals()):
    centroid_check(dataImporter_CY5, CY5_volumeProperty, base_filename+folder_filename+str(experiment)+"_CY5_Data.txt")
    print("CY5 Done")
else:
    print("No CY5 channel")

## Export the data as a single .xlsx file

When you are ready to begin plotting your results. The data from individual experiments needs to be collected into a single file. 

In [ ]:
def restructure_data(path, cell_number, ds, data_set):     
    """
    path  : path to load the individual experiment measurements
    cell_number   : counter to keep track of cell labels
    ds   : experiment information including the cell types involved
    data_set   : output returns the formated data set to be appended with other experiments
    """

    with open(GFP_file, "rb") as fp:   # Unpickling
        object_data_GFP = pickle.load(fp)

    cell_row = []
    for obj in object_data_GFP:
        cell_row = [ds[0][0], "GFP", ds[0][1], ds[0][2], ds[0][3], ds[0][4], ds[0][5], ds[0][6], obj[2], obj[4], obj[6], obj[8], obj[9]]
        data_set.append(cell_row)
        cell_number = cell_number + 1
    return cell_number, data_set

In [ ]:
#list the experiments and initialize local variables
experiments = (1419, 1419)
data_set = []
cell_number = 1

#Get the experimental information from experiment tracker
DOE_file = "Experiment_tracker.xlsx"
if os.path.isfile(base_filename+DOE_file):
    df = pd.read_excel(open(base_filename+DOE_file,'rb'), sheet_name='Experiment_Values')
    df = df[['ID','Cancer cell type', 'Endothelial cell type', 'Biological replicate', 'Technical replicate', 'Hours', 'Condition']]

#Save out matched data
print("Experiment:")
for exp in experiments:
    ds = df[df['ID']==exp].to_numpy()
    print(df[df['ID']==exp])
    folder_filename = "Experiment_"+str(exp).zfill(3)+"/Confocal/"
    file_type = ".txt"
    image_name_RFP = str(exp)+"_RFP_Data"
    image_name_GFP = str(exp)+"_GFP_Data"
    image_name_Cy5 = str(exp)+"_Cy5_Data"
    image_name_DAPI = str(exp)+"_DAPI_Data"
    
    GFP_file = (base_filename+folder_filename+image_name_GFP+file_type)
    if os.path.isfile(GFP_file): 
        cell_number, data_set = restructure_data(GFP_file, cell_number, ds, data_set)
        print("GFP done.")
    Cy5_file = (base_filename+folder_filename+image_name_Cy5+file_type)
    if os.path.isfile(CY5_file):
        cell_number, data_set = restructure_data(GFP_file, cell_number, ds, data_set)
    DAPI_file = (base_filename+folder_filename+image_name_DAPI+file_type)
    if os.path.isfile(DAPI_file):
        cell_number, data_set = restructure_data(GFP_file, cell_number, ds, data_set)
    
#Define how to save out the data into the excel sheet
from openpyxl import load_workbook
path = base_filename + "Data.xlsx"
data_set_df = pd.DataFrame(data_set)
data_set_df.columns = ['ID','Measured Channel','Cancer cell type','Brain niche cell type','Biological replicate','Technical replicate','Hours','Condition','Volume', '% Extravasated','Sphericity','Bounding Box Ratio','Extravasated distance']
data_set_df.to_excel(path, sheet_name = ("Results"))

# Example data plot

## Distance extravasated strip plot

This code contains an example plot but no example data is provided.

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
# the size of A4 paper
fig.set_size_inches(4.3, 3.0)

df = pd.read_excel(base_filename + "Data.xlsx")

#Select experiment by condition or ID or cell type
df = df[(df['Condition']== 1) | (df['Condition']== 2) | (df['Condition']== 3) | (df['Condition']== 4)]
df = df[(df['Measured Channel'] == 'GFP')]

#plot by timepoint
sns.stripplot(x='Hours',y='Extravasated distance',hue='Cancer cell type', hue_order =["MDA-MB-231-BR-GFP", "MDA-MB-231-GFP"],
              marker='o',data=df, jitter=0.4, s=0.6, 
              palette= sns.color_palette([sns.xkcd_rgb["light blue"],sns.xkcd_rgb["orangish"]]),
              order=[24, 48], dodge=True,linewidth=0.1,edgecolor="grey")

plt.legend(bbox_to_anchor=(0.45, 1.30), loc='lower center', ncol=2, fontsize =8)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] + ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(12)

plt.title('Cancer Positions in Astrocyte cell BBNiche', y=1.1, fontweight='bold')
plt.ylabel('Distance Extravasated' u' (\u03bcm)')
plt.xlabel('Exposure to BBNiche (Hours)')
sns.set_style("white")
sns.set_style("ticks")
sns.despine()

plt.gca().set_ylim(-100,200)
plt.subplots_adjust(left = 0.2, bottom = 0.2,top = .7)

fig.savefig(base_filename + "Example_extravasation_plot.png" ,dpi=300, transparent=True)
plt.show()